In [37]:
# Import Libraries
import pandas as pd

# Read in the data
brands = pd.read_csv('data/brands.csv')
finance = pd.read_csv('data/finance.csv')
info = pd.read_csv('data/info.csv')
reviews = pd.read_csv('data/reviews.csv')

In [38]:
# View data
brands.head()

,product_id,brand
0,AH2430,NaN
1,G27341,Adidas
2,CM0081,Adidas
3,B44832,Adidas
4,D98205,Adidas


In [39]:
finance.head()

,product_id,listing_price,sale_price,discount,revenue
0,AH2430,NaN,NaN,NaN,NaN
1,G27341,75.99,37.99,0.5,1641.17
2,CM0081,9.99,5.99,0.4,398.93
3,B44832,69.99,34.99,0.5,2204.37
4,D98205,79.99,39.99,0.5,5182.70


In [40]:
info.head()

,product_name,product_id,description
0,NaN,AH2430,NaN
1,Women's adidas Originals Sleek Shoes,G27341,"A modern take on adidas sport heritage, tailor..."
2,Women's adidas Swim Puka Slippers,CM0081,These adidas Puka slippers for women's come wi...
3,Women's adidas Sport Inspired Questar Ride Shoes,B44832,"Inspired by modern tech runners, these women's..."
4,Women's adidas Originals Taekwondo Shoes,D98205,This design is inspired by vintage Taekwondo s...


In [41]:
reviews.head()

,product_id,rating,reviews
0,AH2430,NaN,NaN
1,G27341,3.3,24.0
2,CM0081,2.6,37.0
3,B44832,4.1,35.0
4,D98205,3.5,72.0


In [42]:
# Create one dataframe merging on product_id
merged_df = info.merge(finance, on='product_id')
merged_df = merged_df.merge(reviews, on='product_id')
merged_df = merged_df.merge(brands, on='product_id')
# View results
merged_df.head()

,product_name,product_id,description,listing_price,sale_price,discount,revenue,rating,reviews,brand
0,NaN,AH2430,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Women's adidas Originals Sleek Shoes,G27341,"A modern take on adidas sport heritage, tailor...",75.99,37.99,0.5,1641.17,3.3,24.0,Adidas
2,Women's adidas Swim Puka Slippers,CM0081,These adidas Puka slippers for women's come wi...,9.99,5.99,0.4,398.93,2.6,37.0,Adidas
3,Women's adidas Sport Inspired Questar Ride Shoes,B44832,"Inspired by modern tech runners, these women's...",69.99,34.99,0.5,2204.37,4.1,35.0,Adidas
4,Women's adidas Originals Taekwondo Shoes,D98205,This design is inspired by vintage Taekwondo s...,79.99,39.99,0.5,5182.70,3.5,72.0,Adidas


In [43]:
# Perform some EDA
# Get summary stats
merged_df.describe()

,listing_price,sale_price,discount,revenue,rating,reviews
count,3120.000000,3120.000000,3120.000000,3120.000000,3120.000000,3120.000000
mean,69.719455,60.103035,0.276090,3951.571263,3.265801,41.545513
std,46.586909,41.906852,0.224955,4440.312551,1.392685,31.361668
min,0.000000,4.490000,0.000000,0.000000,0.000000,0.000000
25%,45.990000,29.990000,0.000000,863.890000,2.600000,12.000000
50%,59.990000,45.990000,0.400000,2735.280000,3.500000,39.000000
75%,89.990000,77.990000,0.500000,5345.015000,4.400000,69.000000
max,299.990000,365.000000,0.600000,64203.930000,5.000000,223.000000


In [44]:
# View data types
merged_df.dtypes

product_name      object
product_id        object
description       object
listing_price    float64
sale_price       float64
discount         float64
revenue          float64
rating           float64
reviews          float64
brand             object
dtype: object

In [45]:
# Identify NA values
merged_df.isna().sum()

product_name     59
product_id        0
description      62
listing_price    59
sale_price       59
discount         59
revenue          59
rating           59
reviews          59
brand            59
dtype: int64

In [47]:
# Drop NA values
merged_df.dropna(inplace=True)
# Verify changes
merged_df.isna().sum()

product_name     0
product_id       0
description      0
listing_price    0
sale_price       0
discount         0
revenue          0
rating           0
reviews          0
brand            0
dtype: int64

In [52]:
# Add price labels based on listing_price quartiles 
merged_df['price_label'] = pd.qcut(merged_df['listing_price'], q=4, labels = ['Budget', 'Average', 'Expensive', 'Elite'])

In [53]:
# Group by brand and price_label to get volume and mean revenue
adidas_vs_nike = merged_df.groupby(["brand", "price_label"], as_index=False).agg(
    num_products=("price_label", "count"), 
    mean_revenue=("revenue", "mean")
).round(2)
# View the results
adidas_vs_nike.head()

/tmp/ipykernel_163/2454666775.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  adidas_vs_nike = merged_df.groupby(["brand", "price_label"], as_index=False).agg(


,brand,price_label,num_products,mean_revenue
0,Adidas,Budget,574,2015.68
1,Adidas,Average,655,3035.30
2,Adidas,Expensive,759,4621.56
3,Adidas,Elite,587,8302.78
4,Nike,Budget,357,1596.33


In [55]:
# Store the length of each description
merged_df['description_length'] = merged_df['description'].str.len()

# Upper description length limits
lengths = [0, 100, 200, 300, 400, 500, 600, 700]

# Description length labels
labels = ['100', '200', '300', '400', '500', '600', '700']

# Cut into bins
merged_df['description_length'] = pd.cut(merged_df['description_length'], bins = lengths, labels = labels)

# Group by bins
description_lengths = merged_df.groupby('description_length', as_index=False).agg(mean_rating=('rating','mean'),total_reviews=('reviews','sum')).round(2)

# View results
description_lengths

/tmp/ipykernel_163/1423079530.py:14: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  description_lengths = merged_df.groupby('description_length', as_index=False).agg(mean_rating=('rating','mean'),total_reviews=('reviews','sum')).round(2)


,description_length,mean_rating,total_reviews
0,100,2.26,36.0
1,200,3.19,17719.0
2,300,3.28,76115.0
3,400,3.29,28994.0
4,500,3.35,4984.0
5,600,3.12,852.0
6,700,3.65,818.0
